In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [9]:

def read_csv(file_path):
  df = pd.read_csv(file_path)
  X = np.array(df["sentence"])
  Y = np.array(df["label"])
  return X,Y

In [10]:

X_train , Y_train = read_csv("/content/drive/MyDrive/dataset/Emoji_Text_Classification/train.csv")
X_test , Y_test = read_csv("/content/drive/MyDrive/dataset/Emoji_Text_Classification/test.csv")


In [11]:
index = 7
X_train[index], Y_train[index]

('congratulations on your acceptance', 2)

In [12]:

def label_to_emoji(label):
    emojies = ["❤️", "🏐", "😄", "😞", "🍴"]
    return emojies[label]

index = 7
print(X_train[index], label_to_emoji(Y_train[index]))


congratulations on your acceptance 😄


In [13]:
len(np.unique(Y_train))
unique,counts = np.unique(Y_train, return_counts=True)
print(unique)
print(counts)

[0 1 2 3 4]
[22 19 38 36 17]


In [14]:

max_len = len(max(X_train , key=len).split(" "))

print(max_len)


10


In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip -d /content/drive/MyDrive/dataset/glov.6b

In [35]:
f = open("/content/drive/MyDrive/dataset/glov.6b/glove.6B.300d.txt", encoding="utf_8")


In [36]:
word_vectoros = {}
for line in f:
  line = line.strip().split()
  word = line[0]
  vector= np.array(line [1:], dtype=np.float64)
  word_vectoros[word] = vector


In [ ]:
word_vectoros["programming"]


In [45]:
def senteence_to_avg(sentence):
  try:
      sentence = sentence.lower()
      words = sentence.strip().split(" ")
      sum_vectors = np.zeros((300, ))
      for word in words:
        sum_vectors += word_vectoros[word]

      avg_vectors = sum_vectors / len(words)
      return avg_vectors

  except:
    print(sentence)
    return None


In [ ]:
senteence_to_avg("i love programming too much")


In [47]:
X_train_avg = []
X_test_avg = []
for x_train in X_train:
  X_train_avg.append(senteence_to_avg(x_train))

X_train_avg = np.array(X_train_avg)

print(X_train_avg.shape)


X_test_avg = []
for x_test in X_test:
    X_test_avg.append(senteence_to_avg(x_test))


X_test_avg = np.array(X_test_avg)
print(X_test_avg.shape)


(132, 300)
(55, 300)


In [ ]:

Y_train_on_hot = tf.keras.utils.to_categorical(Y_train,num_classes=5)
Y_test_on_hot = tf.keras.utils.to_categorical(Y_test, num_classes=5)

Y_train_on_hot
Y_test_on_hot



In [49]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, input_shape=(50,),activation='softmax')

])

In [50]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [51]:
model.fit(X_train_avg , Y_train_on_hot, validation_data=(X_test_avg,Y_test_on_hot),epochs=300)

Epoch 1/300
5/5 [==============================] - 1s 46ms/step - loss: 1.6240 - accuracy: 0.2500 - val_loss: 1.5802 - val_accuracy: 0.2727
Epoch 2/300
5/5 [==============================] - 0s 14ms/step - loss: 1.5554 - accuracy: 0.2803 - val_loss: 1.5481 - val_accuracy: 0.2727
Epoch 3/300
5/5 [==============================] - 0s 14ms/step - loss: 1.5583 - accuracy: 0.2879 - val_loss: 1.5211 - val_accuracy: 0.3091
Epoch 4/300
5/5 [==============================] - 0s 14ms/step - loss: 1.5144 - accuracy: 0.3788 - val_loss: 1.4976 - val_accuracy: 0.3091
Epoch 5/300
5/5 [==============================] - 0s 16ms/step - loss: 1.4764 - accuracy: 0.3485 - val_loss: 1.4764 - val_accuracy: 0.3273
Epoch 6/300
5/5 [==============================] - 0s 15ms/step - loss: 1.4557 - accuracy: 0.3712 - val_loss: 1.4573 - val_accuracy: 0.3273
Epoch 7/300
5/5 [==============================] - 0s 19ms/step - loss: 1.4327 - accuracy: 0.3485 - val_loss: 1.4384 - val_accuracy: 0.3273
Epoch 8/300
5/5 [===

In [ ]:
model.save('/content/drive/MyDrive/dataset/Emoji_Text_Classification/Emoji_Text_Classification_300d.h5')